In [1]:
from preprocessing import SentencesEmbeddingDataset
embedding_type = "glove"
batch_size = 32
NER_dataset = SentencesEmbeddingDataset(embedding_model_type=embedding_type)
train_loader, dev_loader = NER_dataset.get_data_loaders(batch_size=batch_size)

prepering glove...


In [2]:
from model3_comp import LSTM_NER_NN
from train_loop_model3 import train_and_plot_LSTM

In [6]:
from torch.optim import Adam
import torch.nn as nn

num_classes = 2
num_epochs = 10
embedding_dim = NER_dataset.VEC_DIM
lr = 0.001

hidden_dim = 32
# activation = nn.ReLU()
# activation = nn.Sigmoid()
activation = nn.Tanh()
num_layers = 3

model_save_path = (
    f"LSTM_model_stateDict_batchSize_{batch_size}_hidden_{hidden_dim}_lr_{lr}.pt"
)

LSTM_model = LSTM_NER_NN(
    embedding_dim=embedding_dim,
    num_classes=num_classes,
    hidden_dim=hidden_dim,
    model_save_path=model_save_path,
    activation=activation,
    num_layers=num_layers,
)

optimizer = Adam(params=LSTM_model.parameters(), lr=lr)
loss_func = nn.CrossEntropyLoss()

train_and_plot_LSTM(
    LSTM_model=LSTM_model,
    train_loader=train_loader,
    num_epochs=num_epochs,
    val_loader=dev_loader,
    optimizer=optimizer,
    loss_func=loss_func,
)

hidden_dim=32 | activation=Tanh() | num_layers=3
No GPU available, training on CPU.
Batch: 1/107 | Batch Loss: 0.673 | Train Accuracy: 0.956 | Train F1: nan
Batch: 101/107 | Batch Loss: 0.362 | Train Accuracy: 0.950 | Train F1: nan
Epoch: 1/10 | Train Avg Loss: 0.406 | Train Accuracy: 0.950 | Train F1: nan
Val Accuracy: 0.903 | Val F1: nan
Val Accuracy: 0.921 | Val F1: nan
Batch: 1/107 | Batch Loss: 0.378 | Train Accuracy: 0.936 | Train F1: nan
Batch: 101/107 | Batch Loss: 0.352 | Train Accuracy: 0.950 | Train F1: nan
Epoch: 2/10 | Train Avg Loss: 0.364 | Train Accuracy: 0.950 | Train F1: nan
Val Accuracy: 0.927 | Val F1: nan
Val Accuracy: 0.921 | Val F1: nan
Batch: 1/107 | Batch Loss: 0.367 | Train Accuracy: 0.946 | Train F1: nan
Batch: 101/107 | Batch Loss: 0.347 | Train Accuracy: 0.950 | Train F1: nan
Epoch: 3/10 | Train Avg Loss: 0.364 | Train Accuracy: 0.950 | Train F1: nan
Val Accuracy: 0.942 | Val F1: nan
Val Accuracy: 0.921 | Val F1: nan
Batch: 1/107 | Batch Loss: 0.356 | Train